In [2]:
import pymysql

conn = pymysql.connect(host='127.0.0.1', user='root', password='1030', db='zero_pay', charset='utf8')
cur = conn.cursor()

In [3]:
import pandas as pd

In [4]:
# 식권 엑셀파일 -> csv파일로 저장 
# 데이터 가져와서 전처리 
# 전처리 후 상호명만 가져와서 검색하기 

In [5]:
식권 = pd.read_csv('식권.csv', header = 2, usecols = [0,1,4,5,6])
식권.head()

,사용일시,Unnamed: 1,가맹점,가맹점업종,승인금액
0,2023-10-04,11:57:13,미스사이공 고덕푸르지오점,한식 음식점업,"7,000"
1,2023-10-04,11:57:43,버거타임(BURGER TIME),양식,"8,000"
2,2023-10-04,11:58:29,버거타임(BURGER TIME),양식,"8,000"
3,2023-10-04,11:58:31,미스사이공 고덕푸르지오점,한식 음식점업,"8,000"
4,2023-10-04,11:58:44,버거타임(BURGER TIME),양식,"8,000"


In [6]:
식권.columns = ['사용일시','사용시간','식당명','업종','가격']
restau_names = 식권.columns.unique()[2]
restau_names = 식권['식당명'].unique()
restau_names

array(['미스사이공 고덕푸르지오점', '버거타임(BURGER TIME)', '가츠몽상일역점', '덮밥장사장 상일동역점',
       '공차 상일동역점', '김밥천국', '린칭', '이디야고덕교차로', '라홍방마라탕  상일역점',
       '뜨레주르 고덕그라시움점', '이마트24 강일푸르내점', '아비꼬 상일동역점', 'GS25 고덕아르테온점',
       '본도시락 상일점', '죠스떡볶이 앤 바르다김선생 연세상일점', 'GS25 메가스퀘어점',
       '롯데리아문정로데오 주식회사', 'GS25강일리버점', '본 죽&비빔밥 상일동역점', '뚜레쥬르 상일동역',
       '만나냉면', 'KFC 개롱역점', '믿음소망사랑', '이화마라강동본점', '아향', '사누끼',
       '세븐일레븐 고덕리앤파크점', 'GS25 강일9단지점', '상희식당', '홍성원조화로구이', '다이소 상일동역점',
       '쿡희희(cookiehee)', 'GS25중구무학점', 'CU 암사상암로점', '맘마미아김밥',
       '이디야커피 고덕강일2지구점', '방이샤브샤브칼국수', '뚜레주르', 'CU 고덕2단지점', '내가 국수다',
       'GS25강남세곡점', '이삭토스트', '버거타임', '알파쇼핑', 'GS25 상일세종점', '도담한상',
       '메가커피 천호초교사거리점', '순대실록 상일동점', '파리바게트(상일동중앙점)', '프랭크버거 상일점', '마루분식',
       '이마트24 R자곡시티점', 'CU 강남자곡점', '하루엔소쿠자곡점', '차오차오(강일점)', '타피오빈',
       '두레통닭 상일점', '컴포즈커피(개포중앙점)', '우리들은', '또와분식', '굿찬', '써브웨이 상일동점',
       '권사부순대국', '이삭토스트 거여역점', '이마트24 R문정로데오점', 'CU 자곡현대점', '서브웨이 대청점',
       '맘스터치', 'GS25암사역점', '이디야', 'CU 고덕아르테온1상가점', '스탁 (St

In [18]:
import requests
import csv



# 카카오 API 설정
KAKAO_API_KEY = '65cfa5251145011a54255c8c67074e68'  # 카카오 REST API 키

# 카카오 로컬 API를 사용한 장소 검색 함수
def search_places(query):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {
        "query": query,  # 검색할 키워드
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None



def get_all_places(query):
    all_places = []
    for page in range(1):
        data = search_places(query)
        if not data or 'documents' not in data:
            break
        places = data['documents']
        if not places:
            print("No more results.")
            break
        all_places.extend(places)
    return all_places



# 수집된 데이터를 CSV 파일로 저장하는 함수
def save_to_csv(places, filename='zero_pay_restaurant.csv'):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['place_name', 'address_name', 'road_address_name', 'x', 'y']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for place in places:
            writer.writerow({
                'place_name': place.get('place_name'),
                'address_name': place.get('address_name'),
                'road_address_name': place.get('road_address_name'),
                'x': place.get('x'),  # 경도
                'y': place.get('y')   # 위도
            })
    print(f"Saved {len(places)} records to {filename}")

    

# 메인 실행 부분(특정 식당 검색)
if __name__ == "__main__":
    all_places = []
    for (i) in range(len(restau_names)):
        query = f'강동구 {restau_names[i]}'  # 검색 키워드 설정
        places = get_all_places(query)  
        all_places.extend(places) 


    df = pd.DataFrame(all_places)
    df.drop_duplicates(inplace=True)

    save_to_csv(df.to_dict('records'))
    
    

No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more results.
No more result

In [19]:
zero_pay = pd.read_csv('zero_pay_restaurant.csv')
zero_pay.head()

zero_pay['place_name'].isnull().sum()

0

In [20]:
# 식당명과 위도경도만 따로 뽑아 테이블 만듬

map_df = pd.DataFrame({'식당명':zero_pay['place_name'],
                       '경도':zero_pay['x'],
                       '위도':zero_pay['y']})
map_df.head()


,식당명,경도,위도
0,미스사이공 고덕푸르지오점,127.170052,37.557537
1,버거타임,127.170421,37.557594
2,가츠몽 상일역점,127.169327,37.558234
3,덮밥 장사장 상일동역점,127.169678,37.557440
4,공차 상일동역점,127.169609,37.557338


In [21]:
print(map_df[['위도', '경도']].head())
print(map_df.dtypes)

# float 타입으로 변경해야 지도상에 좌표를 찍을 수 있었음
map_df[['위도','경도']].astype(float)
print(map_df.dtypes)

          위도          경도
0  37.557537  127.170052
1  37.557594  127.170421
2  37.558234  127.169327
3  37.557440  127.169678
4  37.557338  127.169609
식당명     object
경도     float64
위도     float64
dtype: object
식당명     object
경도     float64
위도     float64
dtype: object


In [22]:
for i in range(len(map_df)):
    print(f"식당명: {map_df.iloc[i]['식당명']}, 위도: {map_df.iloc[i]['위도']}, 경도: {map_df.iloc[i]['경도']}")

식당명: 미스사이공 고덕푸르지오점, 위도: 37.55753693312697, 경도: 127.1700521823262
식당명: 버거타임, 위도: 37.5575940647314, 경도: 127.17042126747089
식당명: 가츠몽 상일역점, 위도: 37.5582335442234, 경도: 127.169327166229
식당명: 덮밥 장사장 상일동역점, 위도: 37.5574401631642, 경도: 127.16967848081
식당명: 공차 상일동역점, 위도: 37.55733845010448, 경도: 127.16960921275916
식당명: 김밥천국, 위도: 37.5259535335651, 경도: 127.135007863291
식당명: 김밥천국 성내점, 위도: 37.5345657194664, 경도: 127.13305019543
식당명: 신김밥천국, 위도: 37.564832040307, 경도: 127.174548721797
식당명: 김밥천국 길동점, 위도: 37.5401208276461, 경도: 127.144184983934
식당명: 맛있는김밥천국, 위도: 37.5317460175277, 경도: 127.13186403135
식당명: 김밥천국 둔촌2호점, 위도: 37.5296132809453, 경도: 127.142770941972
식당명: 김밥천국 상일점, 위도: 37.5574197140294, 경도: 127.168859038654
식당명: 김밥천국 암사역점, 위도: 37.5499503914436, 경도: 127.127224645344
식당명: 김밥천국, 위도: 37.5404163525404, 경도: 127.147109362989
식당명: 김밥천국 천호점, 위도: 37.5436050974179, 경도: 127.14186521964
식당명: 김밥천국 길동점, 위도: 37.5348135719878, 경도: 127.139997559895
식당명: 김밥천국, 위도: 37.5332826002186, 경도: 127.116967357964
식당명: 포차천국 천호점, 위도: 3

In [23]:
# 식권데이터에서 식당별로 방문횟수 뽑아보기 

식권.head()

,사용일시,사용시간,식당명,업종,가격
0,2023-10-04,11:57:13,미스사이공 고덕푸르지오점,한식 음식점업,"7,000"
1,2023-10-04,11:57:43,버거타임(BURGER TIME),양식,"8,000"
2,2023-10-04,11:58:29,버거타임(BURGER TIME),양식,"8,000"
3,2023-10-04,11:58:31,미스사이공 고덕푸르지오점,한식 음식점업,"8,000"
4,2023-10-04,11:58:44,버거타임(BURGER TIME),양식,"8,000"


In [24]:
방문횟수순위 = 식권.groupby('식당명')['사용일시'].count().sort_values(ascending=False)
방문횟수순위.name = '방문횟수'
방문횟수 = pd.DataFrame(방문횟수순위)

print(방문횟수.columns)
print(방문횟수.dtypes)
print(방문횟수['방문횟수'][0])

Index(['방문횟수'], dtype='object')
방문횟수    int64
dtype: object
195


C:\Users\user\AppData\Local\Temp\ipykernel_16416\3709256518.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(방문횟수['방문횟수'][0])


In [25]:
# map 데이터와 merge
map_df = pd.merge(map_df,방문횟수, on = '식당명')

In [26]:
map_df.head()
map_df = map_df.rename(columns={'count': '방문횟수'})
map_df

,식당명,경도,위도,방문횟수
0,미스사이공 고덕푸르지오점,127.170052,37.557537,170
1,버거타임,127.170421,37.557594,76
2,공차 상일동역점,127.169609,37.557338,24
3,김밥천국,127.135008,37.525954,160
4,김밥천국,127.147109,37.540416,160
...,...,...,...,...
57,복호두 강동상일점,127.168406,37.557363,1
58,늘봄약국,127.118313,37.528076,1
59,버브비,127.172602,37.549125,3
60,CU 천호우성점,127.130201,37.541080,3


In [27]:
방문횟수 = map_df.iloc[0]['방문횟수']
방문횟수

170

In [35]:
# 지도를 그릴때 사용하는 패키지는 folium

import pandas as pd
import folium

# 강동구 중심 좌표로 지도 생성
zero_pay_map = folium.Map(location=[37.5527, 127.1455], zoom_start=13)

# 각 지점에 마커 추가
# popup으로 각 지점별 방문횟수를 추가함
for i in range(len(map_df)):
    try:
        pos_list = [float(map_df.iloc[i]['위도']), float(map_df.iloc[i]['경도'])]
        식당명 = map_df.iloc[i]['식당명']
        방문횟수 = map_df.iloc[i]['방문횟수']
        popup_content = f"방문 횟수: {방문횟수}"
        if 방문횟수 > 100:
            icon_color = 'red'
        elif 방문횟수 > 10:
            icon_color = 'orange'
        elif 방문횟수 < 3 :
            icon_color = 'lightgray'
        else:
            icon_color = 'blue'
        folium.Marker(location=pos_list, popup=folium.Popup(popup_content, max_width = 800), tooltip=식당명,icon=folium.Icon(color=icon_color)).add_to(zero_pay_map)
    except ValueError:
        print(f"Invalid coordinates at index {i}: {map_df.iloc[i]['위도']}, {map_df.iloc[i]['경도']}")

# 지도를 HTML 파일로 저장
zero_pay_map.save('zero_pay.html')